# Setting IDs

There are two ways of setting IDs on text elements in an EpiDoc file:

1. Setting the @xml:id attribute
2. Setting the @n attribute

## Setting the @xml:id attribute

This puts a unique ID on each text element based on its document ID (e.g. 'ISic000001') and the position of this element in the document. This way only works for I.Sicily documents.

In [ ]:
# Load the API and helper functions
from pyepidoc import EpiDoc

In [ ]:
# Set the IDs
doc = EpiDoc('../../examples/ISic000032_untokenized.xml')
print(doc.main_edition.xml_str)


In [ ]:
# Tokenize
doc.tokenize()

# Set the IDs
doc.set_full_ids()
doc.prettify('pyepidoc')
print(doc.main_edition.xml_str)

## Setting the @n attribute

This puts an @n attribute on <w> and <orig> elements. This is an ID local to the document, and should work on any EpiDoc file that contains these elements.

In [ ]:
doc = EpiDoc('../../examples/ISic000032_untokenized.xml')
doc.tokenize()
doc.set_local_ids()
print(doc.main_edition.xml_str)

### Add `@n` IDs to documents that only have `@xml:id`s

In [ ]:
from pyepidoc import EpiDoc, EpiDocCorpus
from pyepidoc.tei.metadata.change import Change
from pyepidoc.tei.metadata.resp_stmt import RespStmt
from pathlib import Path

Load the corpus:

In [ ]:
corpus_path = Path('../../example_corpus')
corpus = EpiDocCorpus(corpus_path)
print('Corpus loaded')

Create metadata elements:

In [ ]:
change = Change.from_details("2025-10-15", "#JB", "Joe Bloggs added local IDs to edition elements")
resp = RespStmt.from_details("Joe Bloggs", "JB", "", "Setting IDs")

Set some `@xml:id`s (for illustrative purposes):

In [ ]:
print('@xml:id set in: ')
for doc in corpus:
    doc.set_full_ids()
    print(f'{doc.id}', end = '; ')

Loop through the documents in the corpus. 
Create local (`@n`) IDs in documents which have `@xml:id`s, but no local IDs:

In [ ]:
print('The following documents have @xml:id IDs but not @n ids')
for doc in corpus:
    if doc.main_edition:
        if doc.main_edition.has_xml_ids:
            print(doc.id)

Set the local ids:

In [ ]:
for doc in corpus:
    if doc.main_edition and doc.main_edition.has_xml_ids:
        if len(doc.local_idable_elements) > 0:
            if not doc.main_edition.has_local_ids:
                print(f'Setting local IDs for {doc.id}')

                # Set the IDs
                doc.set_local_ids()
                
                # Append metadata elements to the document
                doc.append_change(change)
                doc.append_resp_stmt(resp)

                # Prettify the XML
                doc.prettify()

                # Save to file
                doc.to_xml_file(corpus_path / (doc.id + ".xml"), overwrite_existing=True)